In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_cendus_poi = pd.read_pickle('../clean_pickles/target_cendus_poi.pk')
df_cendus_poi_sub = pd.read_pickle('../clean_pickles/target_cendus_poi_sub.pk')
df_dianping_poi = pd.read_pickle('../clean_pickles/target_dianping_poi.pk')
df_dianping_poi_cat = pd.read_pickle('../clean_pickles/target_dianping_poi_cat.pk')
df_mobike = pd.read_pickle('../clean_pickles/target_mobike.pk')
df_mobike_byhours = pd.read_pickle('../clean_pickles/target_mobike_byhours.pk')
df_re = pd.read_pickle('../clean_pickles/target_real_estate.pk')
df_taxi = pd.read_pickle('../clean_pickles/target_taxi_speed.pk')
df_weibo = pd.read_pickle('../clean_pickles/target_weibo.pk')

In [3]:
df_pred = df_cendus_poi.iloc[:,9:len(df_cendus_poi.columns)]
df_pred = df_pred.join(df_cendus_poi_sub.iloc[:,9:len(df_cendus_poi_sub.columns)], rsuffix='_sub')
df_pred = df_pred.join(df_dianping_poi.iloc[:,9:len(df_dianping_poi.columns)])
df_pred = df_pred.join(df_dianping_poi_cat.iloc[:,9:len(df_dianping_poi_cat.columns)])
df_pred = df_pred.join(df_mobike.iloc[:,9:len(df_mobike.columns)])
df_pred = df_pred.join(df_mobike_byhours.iloc[:,9:len(df_mobike_byhours.columns)])
df_pred = df_pred.join(df_re.iloc[:,9:len(df_re.columns)])
df_pred = df_pred.join(df_taxi.iloc[:,9:len(df_taxi.columns)])
df_pred = df_pred.join(df_weibo.iloc[:,9:len(df_weibo.columns)])

### Number of criterias

In [4]:
len(df_pred.columns)

149

In [5]:
def cleanCriteriasNoiseSmell(df):
    droppingColumns = []
    nb0 = df.iloc[:,len(df.columns)-1].value_counts()[0]
    nb1 = df.iloc[:,len(df.columns)-1].value_counts()[1]
    ratio = nb1 / nb0
    counter = 0
    for i in range(0,len(df.columns)-1):
        # 10 NA max to remove a maximum of 3 % of the points
        if len(df[df.iloc[:,i].isna()]) > 10:
            nbi0 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[0]
            try:
                nbi1 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[1]
            except KeyError:
                nbi1 = 0
            #if the proportion of the class is kept then we keep the criterion
            if (nbi1/nbi0) > ratio:
                droppingColumns.append(df.iloc[:,i].name)
                counter +=1
                print(df.iloc[:,i].name,  len(df[df.iloc[:,i].isna()]))
    print(counter)
    return df.drop(columns=droppingColumns)
    

In [6]:
def cleanCriteriasClean(df):
    droppingColumns = []
    nb1 = df.iloc[:,len(df.columns)-1].value_counts()[1]
    nb2 = df.iloc[:,len(df.columns)-1].value_counts()[2]
    nb3 = df.iloc[:,len(df.columns)-1].value_counts()[3]
    nb4 = df.iloc[:,len(df.columns)-1].value_counts()[4]
    ratio = (nb1+nb4) / (nb2+nb3)
    counter = 0
    for i in range(0,len(df.columns)-1):
        # 10 NA max to remove a maximum of 3 % of the points
        if len(df[df.iloc[:,i].isna()]) > 10:
            try:
                nbi1 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[1]
            except KeyError:
                nbi1 = 0
            try:
                nbi2 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[2]
            except KeyError:
                nbi2 = 0
            try:
                nbi3 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[3]
            except KeyError:
                nbi3 = 0
            try:
                nbi4 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[4]
            except KeyError:
                nbi4 = 0
            #if the proportion of the class is kept then we keep the criterion
            if ( (nbi1+nbi4)/(nbi2+nbi3) ) > ratio:
                droppingColumns.append(df.iloc[:,i].name)
                counter +=1
                print(df.iloc[:,i].name,  len(df[df.iloc[:,i].isna()]))
    print(counter)
    return df.drop(columns=droppingColumns)

We display the criteria that have been removed

In [7]:
df_pred_clean = df_pred.join(df_cendus_poi['clean'])
df_pred_clean = df_pred_clean.dropna(subset=['clean'])
df_pred_clean = cleanCriteriasClean(df_pred_clean)
df_pred_clean = df_pred_clean.dropna()
df_pred_clean = df_pred_clean.sample(n=len(df_pred_clean), random_state=3)

av_price_dianping 18
av_decoration_score_dianping 19
av_product_score_dianping 19
av_service_score_dianping 19
av_mobike_duration 47
av_mobike_distance 47
av_mobike_speed 47
av_friends_count_weibo 68
av_followers_count_weibo 68
av_status_count_weibo 68
av_total_checkin_count_weibo 68
av_tip_count_weibo 68
12


In [8]:
df_pred_noise = df_pred.join(df_cendus_poi['noise'])
df_pred_noise = df_pred_noise.dropna(subset=['noise'])
df_pred_noise = cleanCriteriasNoiseSmell(df_pred_noise)
df_pred_noise = df_pred_noise.dropna()
df_pred_noise = df_pred_noise.sample(n=len(df_pred_noise), random_state=3)

av_price_dianping 16
av_decoration_score_dianping 18
av_product_score_dianping 18
av_service_score_dianping 18
av_mobike_duration 39
av_mobike_distance 39
av_mobike_speed 39
taxi_speed_mean 12
av_friends_count_weibo 60
av_followers_count_weibo 60
av_status_count_weibo 60
av_total_checkin_count_weibo 60
av_tip_count_weibo 60
13


In [9]:
df_pred_smell = df_pred.join(df_cendus_poi['smell'])
df_pred_smell = df_pred_smell.dropna(subset=['smell'])
df_pred_smell = cleanCriteriasNoiseSmell(df_pred_smell)
df_pred_smell = df_pred_smell.dropna()
df_pred_smell = df_pred_smell.sample(n=len(df_pred_smell), random_state=3)

av_price_dianping 18
av_decoration_score_dianping 18
av_product_score_dianping 18
av_service_score_dianping 18
av_mobike_duration 43
av_mobike_distance 43
av_mobike_speed 43
av_onesquaremeter_re 16
av_bedroom_re 16
taxi_speed_mean 12
av_friends_count_weibo 63
av_followers_count_weibo 63
av_status_count_weibo 63
av_total_checkin_count_weibo 63
av_tip_count_weibo 63
15


# Prediction

## Noise

In [68]:
X = df_pred_noise.iloc[:,0:len(df_pred_noise.columns)-1].copy()
#X = df_pred_noise[['nb_shopping_cendus']].copy()
Y = df_pred_noise['noise'].copy()

No need to scale value, Adaboost is not sensible

In [65]:
#from sklearn.preprocessing import MinMaxScaler
#X = MinMaxScaler().fit_transform(X)

In [11]:
Y.value_counts()

0.0    344
1.0     87
Name: noise, dtype: int64

Proportion of the class 0:

In [12]:
prop = Y.value_counts()
prop[0.0] / (prop[0.0] + prop[1.0]) * 100

79.81438515081206

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.33, shuffle=False)

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

In [69]:
clf = AdaBoostClassifier(n_estimators = 20)
scores = cross_val_score(clf, X, Y, cv=5)
np.mean(scores)

0.7610264635124299

In [16]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [3,5,8,10,20,30,40,60,80]}
search = GridSearchCV(AdaBoostClassifier(), param_grid, cv=5)
search.fit(X,Y)
search.best_params_

{'n_estimators': 3}

In [17]:
clf = AdaBoostClassifier(n_estimators = 3)
scores = cross_val_score(clf, X, Y, cv=5)
np.mean(scores)

0.7911788291900562

In [18]:
clf = AdaBoostClassifier(n_estimators = 3)
clf = clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_val)
np.mean(Y_pred == Y_val)

0.7762237762237763

In [20]:
from sklearn.metrics import f1_score
f1_score(Y_val, Y_pred)

0.0

In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_val, Y_pred)

array([[111,   2],
       [ 30,   0]], dtype=int64)

## Clean

In [22]:
X = df_pred_clean.iloc[:,0:len(df_pred_clean.columns)-1].copy()
#X = df_cendus_noise[['nb_shopping_cendus']].copy()
Y = df_pred_clean['clean'].copy()

In [23]:
Y.value_counts()

3    218
2    182
4     17
1     15
Name: clean, dtype: int64

Proportion of the class 2 & 3:

In [24]:
prop = Y.value_counts()
(prop[2.0]+prop[3.0]) / (prop[1.0] + prop[2.0] + prop[3.0] + prop[4.0]) * 100

92.5925925925926

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.33, shuffle=False)

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators = 20)
scores = cross_val_score(clf, X, Y, cv=5)
np.mean(scores)

0.4327452552793371

In [29]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [3,5,8,10,20,30,40,60,80]}
search = GridSearchCV(AdaBoostClassifier(), param_grid, cv=5)
search.fit(X,Y)
search.best_params_

{'n_estimators': 8}

In [30]:
clf = AdaBoostClassifier(n_estimators = 8)
scores = cross_val_score(clf, X, Y, cv=5)
np.mean(scores)

0.4654103180967656

In [31]:
clf = AdaBoostClassifier(n_estimators = 8)
clf = clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_val)
np.mean(Y_pred == Y_val)

0.4125874125874126

In [32]:
from sklearn.metrics import f1_score
f1_score(Y_val, Y_pred, average='micro')

0.4125874125874126

In [33]:
from sklearn.metrics import f1_score
f1_score(Y_val, Y_pred, average='weighted')

0.42375832299833616

In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_val, Y_pred)

array([[ 0,  0,  2,  0],
       [ 2, 27, 27,  7],
       [ 1, 37, 31,  3],
       [ 2,  1,  2,  1]], dtype=int64)

In [35]:
Y_val.value_counts()

3    72
2    63
4     6
1     2
Name: clean, dtype: int64

In [36]:
pd.Series(Y_pred).value_counts()

2    65
3    62
4    11
1     5
dtype: int64

## Smell

In [37]:
X = df_pred_smell.iloc[:,0:len(df_pred_smell.columns)-1].copy()
#X = df_cendus_noise[['nb_shopping_cendus']].copy()
Y = df_pred_smell['smell'].copy()

In [38]:
Y.value_counts()

0.0    367
1.0     16
Name: smell, dtype: int64

Proportion of the class 0:

In [39]:
prop = Y.value_counts()
prop[0.0] / (prop[0.0] + prop[1.0]) * 100

95.822454308094

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.33, shuffle=True)

In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators = 20)
scores = cross_val_score(clf, X, Y, cv=5)
np.mean(scores)

0.9451127819548872

In [42]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [3,5,7,10,20,30,40,60,80]}
search = GridSearchCV(AdaBoostClassifier(), param_grid, cv=5)
search.fit(X,Y)
search.best_params_

{'n_estimators': 3}

In [43]:
clf = AdaBoostClassifier(n_estimators = 3)
scores = cross_val_score(clf, X, Y, cv=5)
np.mean(scores)

0.9530075187969926

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.33, shuffle=True)
clf = AdaBoostClassifier(n_estimators = 3)
clf = clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_val)
np.mean(Y_pred == Y_val)

0.968503937007874

In [59]:
from sklearn.metrics import f1_score
f1_score(Y_val, Y_pred, average='binary')

0.0

In [60]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_val, Y_pred)

array([[123,   0],
       [  4,   0]], dtype=int64)

In [61]:
Y_val.value_counts()

0.0    123
1.0      4
Name: smell, dtype: int64

In [62]:
pd.Series(Y_pred).value_counts()

0.0    127
dtype: int64